### Testing HCP Queue 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 40]

In [3]:
import radio as rio
from radio.data.datautils import get_first_batch, create_probability_map
from radio.data.datavisualization import plot_batch

#### Setup Data

In [6]:
# Load pre-stored data from train/test/val folders 
data = rio.data.HCPPatchDataModule(
    root="/media/cerebro/Workspaces/Students/Eduardo_Diniz/Studies",
    study="HCP_test",
    subj_dir="radio_3T_MPR/unprocessed",
    data_dir="",
    use_augmentation=False,
    use_preprocessing=True,
    resample=False,
    batch_size=4,
    modalities=["3T_MPR"],
    labels=[],
    dims=None,
    patch_size=(256, 320, 1),
    create_custom_probability_map=True,
    queue_max_length=300,
    samples_per_volume=10,
)
data.prepare_data()
data.setup(stage='fit')

In [7]:
print(f"Training: {data.size_train} patches.")
print(f"Validation: {data.size_val} patches.")

Training: 70 patches.
Validation: 10 patches.


#### Get a Train and a Test Batch

In [ ]:
train_dataloader = data.train_dataloader()
val_dataloader = data.val_dataloader()

In [ ]:
# Takes a long time if resampling is required.
# Preprocessing beforehand is therefore recommended to speed up training.
train_batch = get_first_batch(train_dataloader)
val_batch = get_first_batch(val_dataloader)

In [ ]:
print(f"Batch Keys: {train_batch.keys()}")
print(f"3T_MPR Shape: {train_batch['3T_MPR']['data'].shape}")
print(f"3T_MPR keys: {train_batch['3T_MPR'].keys()}")
print(f"Subject IDs: {train_batch['subj_id']}")
print(f"Location Shape: {train_batch['location'].shape}")

#### Plot Example Samples from Batches

In [ ]:
plot_batch(train_batch)